In [1]:
import itertools
import pandas as pd
import scipy.interpolate as inter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.signal as sig
import scipy.misc as misc
import scipy.integrate as integrate
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import erfc

def mintime(time):
    return 5*np.ceil(time/5.0)

def maxtime(time):
    return 5*np.floor(time/5.0)

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = sig.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = sig.filtfilt(b, a, data)
    return y

def partial_derivative2d(function, x, y, dx, dy):
    value1 = function(x-dx/2, y-dy/2)
    value2 = function(x+dx/2, y+dy/2)
    return (value2-value1)/np.sqrt(dx**2+dy**2)
    
def CloughTocher2d_interpolator(xref, yref, vals):
    """ 2d interpolation (requires Scipy).
     
    This is a convenience wrapper around
    scipy.interpolation.CloughTocher2Dinterpolator, that saves you
    having to worry about using meshgrid.
 
    Parameters
    ----------
    xref, yref : array of floats, shapes (J,), (I,)
      Reference coordinate grid. The grid must be equally spaced along
      each direction, but the spacing can be different between
      directions.
    vals : array of floats, shape (I, J)
      Reference values at the reference grid positions.
 
    Returns
    -------
    interpolator: CloughTocher2DInterpolater instance
      Object that accepts a (y,x) tuple (note reversed order from the
      input to this function!) and returns the interpolated value.
 
    See Also
    --------
    barak.plot.arrplot for plotting the reference and interpolated arrays.
    """
 
    assert (len(yref), len(xref)) == vals.shape
    XREF,YREF = np.meshgrid(xref, yref)
    
    interpolator = inter.CloughTocher2DInterpolator((XREF.ravel(), YREF.ravel()),
                                              vals.ravel())
 
    return interpolator

def constantfluxsurface(x, t, params):
    F0 = params[0]
    k = params[1]
    kappa = params[2]
    temps = 2*F0/2*((kappa*t/np.pi)**0.5*np.exp(-x/(4*kappa*t))-x/2*erfc(x/(2*kappa *t)**0.5))
    return temps

def residual(x, t, params, measured_temp):
    calculated_temps = consantfluxsurface(x, t, params)
    return temps - calculated_temps
    
    

    
order = 3
fs = 10.0   # sample rate, Hz
cutoff = 0.025# desired cutoff frequency of the filter, Hz



In [2]:
# load all the data into the script and plot a key temperature to decide the start and end of the heat flux experiment
%matplotlib ipympl

filenames = ["Conv150_1_1","Conv150_1_2","Conv150_2_1","Conv150_2_2","Conv150_3_1","Conv150_3_2","Conv150_4_1","Conv150_4_2",
"Conv150_6_1","Conv150_6_2",
"Conv250_1_1","Conv250_1_2","Conv250_2_1","Conv250_2_2","Conv250_3_1","Conv250_3_2","Conv250_4_1","Conv250_4_2","Conv250_6_1",
"Conv200_1_1","Conv200_1_2","Conv200_2_1","Conv200_3_1","Conv200_3_2","Conv200_4_1","Conv200_4_2","Conv200_6_1","Conv200_6_2",
"LowE200_1_1","LowE200_1_2","LowE200_2_1","LowE200_2_2","LowE200_3_1", "LowE200_3_2","LowE200_4_1","LowE200_4_2","LowE200_6_1","LowE200_6_2"]

dfs = []

test_TCs = ["T23RAW", "T19RAW"]
figs = []

for i in range(len(filenames)):
    figs.append(plt.subplots(nrows=1))
    fig, ax = figs[i]
    
    filename = filenames[i]
    df = pd.read_feather("Processed//"+filename+'.feather')
    dfs.append(df)
    colors = itertools.cycle(cm.rainbow(np.linspace(0, 1, len(test_TCs))))
    
    if filename[0:4] == "Conv":
        df["emissivity"] = 1.0
        df["ali thickness"] = 0.002
    else:
        df["emissivity"] = 0.05
        df["ali thickness"] = 0.003
    df['jet temp'] = int(filename[5:7])
    df['position'] = int(filename[8])
    df['repeat'] = int(filename[10])
    for TC in test_TCs:
        c = next(colors)
        ax.plot(df["time"],df[TC], label = TC, color = c)
    ax.set_title(filenames[i])
    fig.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The above plot is used to ascertain the start and end of each of the heat flux experiments, this data is then inputted in the "start_ends" list below which crops the data to this range

In [3]:
i=0


start_ends = [[100,520],[80,410],
              [80,860],[80,860],
              [80,315],[85,415],
              [85,510],[75,755],
              [75,760],[100,600], 
              [80,215],[80,245],
              [50,180],[60,190],
              [55,190],[55,230],
             [70,250],[65,225],
             [75,275],
             [65,345],[75,320],
             [55,315],
             [80,370],[45,355],
              [80,1140],[30,400],
              [170,425],[70,340],
              [140,550],[85,450],
              [70,475],[70,565],
             [70,560], [80,515],
             [70,475], [65,675],
              [55,410],[110,500]]


for i in range(len(dfs)):
    dfs[i]['time'] = dfs[i]['time'] - start_ends[i][0]
    dfs[i] = dfs[i][dfs[i]['time']<start_ends[i][1]-start_ends[i][0]]
    dfs[i] = dfs[i][dfs[i]['time']>=0]
    #dfs[i] = dfs[i].sort_index()

The flux data is calculated using a function copied from the master data processing script on github, flux and HTC columns are added to the dataframes

In [4]:
# copied from master processing script

import pandas as pd
import scipy.interpolate as inter
import scipy.integrate as integrate

def heat_flux_sensor_calculation(df, sensor_information, boundary_TC):

    for TC in sensor_information[1]:
        df[TC+' change'] = df[TC] - df.iloc[0][TC]    
    perspex_energy = []
    for time in df.index:
        temp_interp = inter.UnivariateSpline(sensor_information[0], 
                                             [df.at[time, x+' change'] for x in sensor_information[1]], s=0)
        perspex_energy.append(1190*1466*integrate.quad(temp_interp, a=0, b=0.02)[0])
    df['perspex energy'] = perspex_energy
    df['ali energy'] = df[sensor_information[1][0]+' change']*2710.0*910.0*df["ali thickness"]
    df['total energy'] = df['perspex energy']+df['ali energy']
    df['flux'] = df['total energy'].diff(1)/5.0
    df['HTC'] = df['flux']/(df[boundary_TC] - df[sensor_information[1][0]])
    return df


sensor_information = [[0.0, 0.005, 0.01, 0.015, 0.02], ['T19RAW', 'T18RAW', 'T20RAW', 'T16RAW', 'T17RAW']]
#                      location of TC in mm from the surface, name of TC
boundary_TC = 'T23'# TC in air boundary layer
for df in dfs:
    df = heat_flux_sensor_calculation(df, sensor_information, boundary_TC)  

The flux data is then fitted to a function of flux,this can be editted in the "surface_flux" function.

Once fitted a "calc flux" column is added to the dataframe

Additionally a results list is created which stores the output from the fitting function

In [5]:
def surface_flux(params, df, sensor_information, boundary_TC):
    convective_flux = params[0]*(df[boundary_TC]-df[sensor_information[1][0]])
    radiative_flux = df["emissivity"]*5.670373*10**-8*((params[2]+273.15)**4.0-
                                        (df[sensor_information[1][0]]+273.15)**4.0)
    return convective_flux+radiative_flux

def residual(params, df, sensor_information, boundary_TC, flux):
    calc_fluxes = surface_flux(params, df, sensor_information, boundary_TC)
    #print(params, df, HeatFluxTCs, boundary_TC, flux)
    return np.sum((calc_fluxes - df[flux])**2.0)

def fitting_wrapper(df, sensor_information, boundary_TC, flux):
    x0 = [30, 1, 180]
    result = opt.minimize(residual, x0, args=(df, sensor_information, boundary_TC, flux), bounds=bounds)
    return result

bounds = [[0, 100],[0, 1],[100, 350]]

results = []
for df in dfs:
    result = fitting_wrapper(df, sensor_information, boundary_TC, 'flux')
    df['calc flux'] = surface_flux(result.x, df, sensor_information, boundary_TC)
    results.append(result)

Printing results from the function fitting

In [29]:
for i in range(len(filenames)):
    name = filenames[i]
    res = results[i].x
    label_string = f' HTC:{res[0]:.2f}, epsilon:{res[1]:.2f}, BB temp:{res[2]:.2f})'
    print(name, label_string)

Conv150_1_1  HTC:20.66, epsilon:1.00, BB temp:157.34)
Conv150_1_2  HTC:23.30, epsilon:1.00, BB temp:157.90)
Conv150_2_1  HTC:31.41, epsilon:1.00, BB temp:125.80)
Conv150_2_2  HTC:31.37, epsilon:1.00, BB temp:125.94)
Conv150_3_1  HTC:21.90, epsilon:1.00, BB temp:161.88)
Conv150_3_2  HTC:18.49, epsilon:1.00, BB temp:159.48)
Conv150_4_1  HTC:22.87, epsilon:1.00, BB temp:122.94)
Conv150_4_2  HTC:32.46, epsilon:1.00, BB temp:100.00)
Conv150_6_1  HTC:37.91, epsilon:1.00, BB temp:131.73)
Conv150_6_2  HTC:35.58, epsilon:1.00, BB temp:140.42)
Conv250_1_1  HTC:6.00, epsilon:1.00, BB temp:298.23)
Conv250_1_2  HTC:12.34, epsilon:1.00, BB temp:278.13)
Conv250_2_1  HTC:5.85, epsilon:1.00, BB temp:295.00)
Conv250_2_2  HTC:7.69, epsilon:1.00, BB temp:286.58)
Conv250_3_1  HTC:0.00, epsilon:1.00, BB temp:308.70)
Conv250_3_2  HTC:0.00, epsilon:1.00, BB temp:300.82)
Conv250_4_1  HTC:18.75, epsilon:1.00, BB temp:231.47)
Conv250_4_2  HTC:22.63, epsilon:1.00, BB temp:220.62)
Conv250_6_1  HTC:31.15, epsilon:1

Plotting for comparing caulcated / fitted heat fluxes with mesaured heat fluxes

In [6]:
%matplotlib ipympl

colors = itertools.cycle(cm.rainbow(np.linspace(0, 1, len(dfs))))
i = 0
figs = []
for df in dfs:
    figs.append(plt.subplots(nrows=1))
    fig, ax = figs[i]
    res = results[i].x
    label_string = f' HTC:{res[0]:.2f}, epsilon:{res[1]:.2f}, BB temp:{res[2]:.2f})'
    #label_string = f' HTC:{res[0]:.2f}'
    ax.plot(df['time'], df['flux'], label = filenames[i]+ label_string, c='k', ls='-')
    ax.plot(df["time"], df['calc flux'], c='r', ls='--')
    i = i+1
    plt.legend()
    plt.xlabel('Time /s')
    plt.ylabel('Surface Flux /W/m2')
    #plt.xlim(0,400)
    #plt.ylim(0,2200)
    plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Conv150_1_1  HTC:37.46, epsilon:1.00, BB temp:250.00)
Conv150_1_2  HTC:40.82, epsilon:1.00, BB temp:250.00)
Conv150_2_1  HTC:41.20, epsilon:1.00, BB temp:217.02)
Conv150_2_2  HTC:41.17, epsilon:1.00, BB temp:217.52)
Conv150_3_1  HTC:39.76, epsilon:1.00, BB temp:250.00)
Conv150_3_2  HTC:36.20, epsilon:1.00, BB temp:250.00)
Conv150_4_1  HTC:29.68, epsilon:1.00, BB temp:250.00)
Conv150_4_2  HTC:38.37, epsilon:0.23, BB temp:245.75)
Conv150_6_1  HTC:51.10, epsilon:1.00, BB temp:245.11)
Conv150_6_2  HTC:48.75, epsilon:1.00, BB temp:250.00)
Conv250_1_1  HTC:39.61, epsilon:0.00, BB temp:189.58)
Conv250_1_2  HTC:45.87, epsilon:0.00, BB temp:195.95)
Conv250_2_1  HTC:44.23, epsilon:1.00, BB temp:250.00)
Conv250_2_2  HTC:41.15, epsilon:1.00, BB temp:250.00)
Conv250_3_1  HTC:40.71, epsilon:1.00, BB temp:250.00)
Conv250_3_2  HTC:40.42, epsilon:1.00, BB temp:250.00)


In [ ]:
results[0]

In [16]:
filenames[0][8]

'1'

In [ ]:
%matplotlib tk
df = dfs[2]
plt.plot(df['time'],df["T23RAW"])
df = dfs[3]
plt.plot(df['time'],df["T23RAW"])
plt.show()

In [20]:
len(start_ends)

31

In [22]:
len(dfs)

32